<div style="display: flex; background-color: RGB(255,114,0);" >
<h1 style="margin: auto; padding: 30px; ">ANALYSE DU STOCK ET DES VENTES DU SITE BOTTLENECK</h1>
</div>

# OBJECTIF DE CE NOTEBOOK

Bienvenue dans l'outil plébiscité par les analystes de données Jupyter.

Il s'agit d'un outil permettant de mixer et d'alterner code, texte et graphiques.

Cet outil est formidable pour plusieurs raisons:

+ Il permet de tester des lignes de codes au fur et à mesure de votre rédaction, de constater immédiatement le résultat d'une instruction, de la corriger si nécessaire.
+ Il permet aussi de rédiger du texte pour expliquer l'approche suivie ou les résultats d'une analyse et de le mettre en forme grâce à du code html ou plus simple avec **Markdown**
+ Il est possible d'ajouter des graphiques

Pour vous aider dans vos premiers pas à l'usage de Jupyter et de Python, nous avons rédigé ce notebook en vous indiquant les instructions à suivre.

Il vous suffit pour cela de saisir le code Python répondant à l'instruction donnée.

Vous verrez de temps à autre le code Python répondant à une instruction donnée mais cela est fait pour vous aider à comprendre la nature du travail qui vous est demandé.

Et gardez à l'esprit qu'il n'y a pas de solution unique pour résoudre un problème et qu'il y a autant de résolutions de problèmes que de développeurs ;)...



<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 1 - Importation des librairies et chargement des fichiers</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.1 - Importation des librairies</h3>
</div>

In [1]:
#Importation de la librairie Pandas
import pandas as pd
import numpy as np

In [2]:
#Importation de la librairie plotly express
import plotly.express as px      
import plotly.graph_objects as go
import openpyxl

In [3]:
#Trouver dans Google l'instruction permettant d'afficher toutes les colonnes d'un dataframe
#Saisir dans Google les mots clés "display all columns dataframe Pandas" par exemple.
#Dans les résultats de la recherche, privilégier les solutions provenant de Stack Overflow ou Medium
pd.set_option('display.max_columns', None)

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.2 - Chargements des fichiers</h3>
</div>

In [4]:
#Importation du fichier web.xlsx
df_web = pd.read_excel("C:/Users/AUCLAIR/Documents/Projet_Data_6/web.xlsx")
#Importation du fichier erp.xlsx
df_erp = pd.read_excel("C:/Users/AUCLAIR/Documents/Projet_Data_6/erp.xlsx")
#Importation du fichier liaison.xlsx
df_liaison = pd.read_excel("C:/Users/AUCLAIR/Documents/Projet_Data_6/liaison.xlsx")

C:\Users\AUCLAIR\Documents\Python\environnement\env_base\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\AUCLAIR\Documents\Python\environnement\env_base\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\AUCLAIR\Documents\Python\environnement\env_base\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 2 - Analyse exploratoire des fichiers</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1 - Analyse exploratoire du fichier erp.xlsx</h3>
</div>

In [5]:
#Afficher les dimensions du dataset
print("Le tableau comporte {} observation(s) ou article(s)".format(df_erp.shape[0]))
print("Le tableau comporte {} colonne(s)".format(df_erp.shape[1]))

Le tableau comporte 825 observation(s) ou article(s)
Le tableau comporte 6 colonne(s)


In [6]:
#Consulter le nombre de colonnes
print("Le DataFrame comporte", df_erp.shape[1], "colonnes")

Le DataFrame comporte 6 colonnes


In [7]:
#La nature des données dans chacune des colonnes
df_erp.dtypes

product_id          int64
onsale_web          int64
price             float64
stock_quantity      int64
stock_status       object
purchase_price    float64
dtype: object

In [8]:
#Le nombre de valeurs présentes dans chacune des colonnes
df_erp.count()

product_id        825
onsale_web        825
price             825
stock_quantity    825
stock_status      825
purchase_price    825
dtype: int64

In [9]:
#Le nombre de valeurs présentes dans chacune des colonnes
df_erp.isna().sum()

product_id        0
onsale_web        0
price             0
stock_quantity    0
stock_status      0
purchase_price    0
dtype: int64

In [10]:
#Afficher les 5 premières lignes de la table
df_erp.head()

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price
0,3847,1,24.2,16,instock,12.88
1,3849,1,34.3,10,instock,17.54
2,3850,1,20.8,0,outofstock,10.64
3,4032,1,14.1,26,instock,6.92
4,4039,1,46.0,3,outofstock,23.77


In [11]:
#Vérifier si il y a des lignes en doublon dans la colonne product_id
doublons_erp = df_erp.duplicated(subset='product_id')
df_erp[doublons_erp]

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price


In [12]:
#Afficher les valeurs distinctes de la colonne stock_status
df_erp['stock_status'].unique()

array(['instock', 'outofstock'], dtype=object)

In [13]:
#À quelle(s) autre(s) colonne(s) sont-elles liées ?
df_erp[(df_erp['stock_status']=='outofstock') & (df_erp['stock_quantity']!=0)]

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price
4,4039,1,46.0,3,outofstock,23.77
449,4973,0,10.0,-10,outofstock,4.96
573,5700,1,44.5,-1,outofstock,22.30


**Je constate que les colonnes stock_quantity et stock_status, sont logiquement liées, les 3 exceptions notées au dessus correspondent certaines à des erreurs.**

In [14]:
#Création d'une colonne "stock_status_2"
#La valeur de cette deuxième colonne sera fonction de la valeur dans la colonne "stock_quantity"
#Si la valeur de la colonne "stock_quantity" est nulle, renseigner "outofstock" sinon mettre "instock"
stock =df_erp["stock_quantity"]
df_erp["stock_status_2"] = np.where(stock==0, 'outofstock', 'instock')
df_erp[["stock_quantity", "stock_status_2"]]

,stock_quantity,stock_status_2
0,16,instock
1,10,instock
2,0,outofstock
3,26,instock
4,3,instock
...,...,...
820,30,instock
821,9,instock
822,6,instock
823,14,instock


In [15]:
#Vérifions que les 2 colonnes sont identiques:
#Les 2 colonnes sont strictement identiques si les valeurs de chaque ligne sont strictement identiques 2 à 2
#La comparaison de 2 colonnes peut se réaliser simplement avec l'instruction ci-dessous:
df_erp["stock_status"] == df_erp["stock_status_2"]

#Le résultat est l'affichage de True ou False pour chacune des lignes du dataset
#C'est un bon début, mais difficile à exploiter

0       True
1       True
2       True
3       True
4      False
       ...  
820     True
821     True
822     True
823     True
824     True
Length: 825, dtype: bool

In [16]:
#Mais il est possible de synthétiser ce résultat en effectuant la somme de cette colonne:
#True vaut 1 et False 0
#Nous devrions obtenir la somme de 824 qui correspond au nombre de lignes dans ce dataset
liste = np.where(df_erp["stock_status"] == df_erp["stock_status_2"], 1, 0)
print("Nombre de lignes identiques :", liste.sum(), "sur 825")

Nombre de lignes identiques : 821 sur 825


In [17]:
#Si les colonnes ne sont absolument pas identiques ligne à ligne alors identifier la ligne en écart
##Dans ce cas je vous donne ce lien pour apprendre à réaliser des filtres dans Pandas:
##https://bitbucket.org/hrojas/learn-pandas/src/master/
##Lesson 3

In [18]:
#Identification des lignes différentes
df_erp[df_erp["stock_status"] != df_erp["stock_status_2"]]

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price,stock_status_2
4,4039,1,46.0,3,outofstock,23.77,instock
398,4885,1,18.7,0,instock,9.66,outofstock
449,4973,0,10.0,-10,outofstock,4.96,instock
573,5700,1,44.5,-1,outofstock,22.30,instock


In [19]:
#Corriger la ou les données incohérentes
df_erp.loc[df_erp['stock_quantity'] <  0, "stock_quantity"] = 0
#Vérification en utilisant le même code que plus haut pour afficher les problèmes
df_erp[df_erp["stock_status"] != df_erp["stock_status_2"]]

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price,stock_status_2
4,4039,1,46.0,3,outofstock,23.77,instock
398,4885,1,18.7,0,instock,9.66,outofstock
449,4973,0,10.0,0,outofstock,4.96,instock
573,5700,1,44.5,0,outofstock,22.30,instock


In [20]:
# Je recréé la colonne "stock_status_2" pour s'adapter au nouveau changement
df_erp["stock_status_2"] = np.where(df_erp['stock_quantity']==0, 'outofstock', 'instock')
df_erp[df_erp["stock_status"] != df_erp["stock_status_2"]]

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price,stock_status_2
4,4039,1,46.0,3,outofstock,23.77,instock
398,4885,1,18.7,0,instock,9.66,outofstock


**Je décide de remplacer les valeurs négatives par 0, mon choix étant appuyé par la valeur 'outofstock' dans la colonne "stock_status". Je relance pour finir la colonne "stock_status_2" pour qu'elle s'adapte à ce changement.** 

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1 - Analyse exploratoire de chaque variable du fichier erp.xlsx</h3>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1.1 - Analyse de la variable PRIX</h3>
</div>

In [21]:
###############
## LES PRIX  ##
###############
#Vérification des prix: Y a t-il des prix non renseignés?
df_erp['price'].isna().sum()

np.int64(0)

In [22]:
#Vérification des prix: Y a t-il des prix  négatifs ou nuls?
df_erp_err_val = df_erp[df_erp['price'] <= 0]
df_erp_err_val

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price,stock_status_2
151,4233,0,-20.0,0,outofstock,10.33,outofstock
469,5017,0,-8.0,0,outofstock,4.34,outofstock
739,6594,0,-9.1,19,instock,4.61,instock


In [23]:
#Afficher le ou les prix non renseignés dans la colonne "price"
print("Nombres d'articles avec un prix non renseigné: {}".format(len(df_erp_err_val))) 
#Afficher le prix minimum de la colonne "price"
print("prix minimum de la colonne 'price': {}".format(df_erp['price'].min()))
#Afficher le prix maximum de la colonne "price"
print("prix minimum de la colonne 'price': {}".format(df_erp['price'].max()))
#Afficher les prix inférieurs à 0 (qu'est-ce qu'il faut en faire ?)
print("Différents prix inférieurs à 0 :")
for p in df_erp_err_val['price']:
    print(f" {p}")

Nombres d'articles avec un prix non renseigné: 3
prix minimum de la colonne 'price': -20.0
prix minimum de la colonne 'price': 225.0
Différents prix inférieurs à 0 :
 -20.0
 -8.0
 -9.1


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1.2 - Analyse de la variable STOCK</h3>
</div>

In [24]:
#######################
### stock_quantity  ###
#######################

#Afficher la quantité minimum de la colonne "stock_quantity"
print(df_erp['stock_quantity'].min())

0


In [25]:
#Afficher la quantité maximum de la colonne "stock_quantity"
print(df_erp['stock_quantity'].max())

145


In [26]:
#Afficher les stocks inférieurs à 0 (qu'est-ce qu'il faut en faire ?)
df_erp[df_erp['stock_quantity']<0]

,product_id,onsale_web,price,stock_quantity,stock_status,purchase_price,stock_status_2


**Je note que aucune ligne (donc un produit précis) ne correspond à une valeur négative, ce qui est rassurant.**

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1.3 - Analyse de la variable ONSALE_WEB</h3>
</div>

In [27]:
#Vérification de la colonne onsale_web et des valeurs qu'elle contient. Que signifient-elles?
df_erp['onsale_web'].value_counts()

onsale_web
1    716
0    109
Name: count, dtype: int64

In [28]:
#Quelles sont les colonnes à conserver selon vous?


In [29]:
#Supprimer la colonne comportant le libellé "stock_status_2" car elle est redondante 
#avec la colonne "stock_status".


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1.1.4 - Analyse de la variable prix d'achat</h3>
</div>

In [30]:
######################
##   prix d'achat   ##
######################

#Vérification de la colonne purchase_price : 
#Afficher le ou les prix non renseignés dans la colonne "purchase_price"
print("Nombre de valeurs manquantes dans la colonne 'purchase_price' :", df_erp['purchase_price'].isna().sum())

Nombre de valeurs manquantes dans la colonne 'purchase_price' : 0


In [31]:
#Afficher le prix minimum de la colonne "purchase_price"
df_erp['purchase_price'].min()

np.float64(2.74)

In [32]:
#Afficher le prix maximum de la colonne "purchase_price"
df_erp['purchase_price'].max()

np.float64(137.81)

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.2 - Analyse exploratoire du fichier web.xlsx</h3>
</div>
 

In [33]:
#Dimension du dataset
df_web.shape

(1513, 29)

In [34]:
#Nombre d'observations
print(f"Nombre d'observations (lignes) : {df_web.shape[0]}")

Nombre d'observations (lignes) : 1513


In [35]:
#Nombre de caractéristiques
print(f"Nombre de caractéristiques (colonnes) : {df_web.shape[1]}")

Nombre de caractéristiques (colonnes) : 29


In [36]:
#La nature des données dans chacune des colonnes
df_web.dtypes

sku                              object
virtual                           int64
downloadable                      int64
rating_count                      int64
average_rating                  float64
total_sales                     float64
tax_status                       object
tax_class                       float64
post_author                     float64
post_date                datetime64[ns]
post_date_gmt            datetime64[ns]
post_content                    float64
product_type                     object
post_title                       object
post_excerpt                     object
post_status                      object
comment_status                   object
ping_status                      object
post_password                   float64
post_name                        object
post_modified            datetime64[ns]
post_modified_gmt        datetime64[ns]
post_content_filtered           float64
post_parent                     float64
guid                             object


In [37]:
#Le nombre de valeurs présentes dans chacune des colonnes
df_web.isna().sum()

sku                        85
virtual                     0
downloadable                0
rating_count                0
average_rating             83
total_sales                83
tax_status                797
tax_class                1513
post_author                83
post_date                  83
post_date_gmt              83
post_content             1513
product_type               84
post_title                 83
post_excerpt              797
post_status                83
comment_status             83
ping_status                83
post_password            1513
post_name                  83
post_modified              83
post_modified_gmt          83
post_content_filtered    1513
post_parent                83
guid                       83
menu_order                 83
post_type                  83
post_mime_type            799
comment_count              83
dtype: int64

In [38]:
df_web.head(3)

,sku,virtual,downloadable,rating_count,average_rating,total_sales,tax_status,tax_class,post_author,post_date,post_date_gmt,post_content,product_type,post_title,post_excerpt,post_status,comment_status,ping_status,post_password,post_name,post_modified,post_modified_gmt,post_content_filtered,post_parent,guid,menu_order,post_type,post_mime_type,comment_count
0,11862,0,0,0,0.0,3.0,NaN,NaN,2.0,2018-02-12 13:46:23,2018-02-12 12:46:23,NaN,Vin,Gilles Robin Hermitage Rouge 2012,NaN,publish,closed,closed,NaN,gilles-robin-hermitage-2012,2019-01-31 12:12:56,2019-01-31 11:12:56,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0
1,16057,0,0,0,0.0,5.0,NaN,NaN,2.0,2018-04-17 15:29:17,2018-04-17 13:29:17,NaN,Vin,Domaine Pellé Sancerre Rouge La Croix Au Garde...,NaN,publish,closed,closed,NaN,pelle-sancerre-rouge-la-croix-au-garde-2017,2020-07-07 10:05:02,2020-07-07 08:05:02,NaN,0.0,https://www.bottle-neck.fr/wp-content/uploads/...,0.0,attachment,image/jpeg,0.0
2,14692,0,0,0,0.0,5.0,taxable,NaN,2.0,2019-03-19 10:06:47,2019-03-19 09:06:47,NaN,Vin,Château Fonréaud Bordeaux Blanc Le Cygne 2016,<div>Grâce à la complémentarité des 3 cépages ...,publish,closed,closed,NaN,fonreaud-bordeaux-blanc-le-cygne-2016,2020-04-25 21:40:31,2020-04-25 19:40:31,NaN,0.0,https://www.bottle-neck.fr/?post_type=product&...,0.0,product,NaN,0.0


In [39]:
#Selon vous, quelles sont les colonnes à conserver ?


In [40]:
#Si vous avez défini des colonnes à supprimer, effectuer l'opération
cols_to_drop = (
    list(df_web.columns[1:5]) +
    list(df_web.columns[7:12]) +
    list(df_web.columns[15:19]) +
    list(df_web.columns[20:24]) +
    [df_web.columns[25]] +
    [df_web.columns[28]]
)
df_web = df_web.drop(cols_to_drop, axis=1)

In [41]:
#Visualisation des valeurs de la colonne sku
#Quelles sont les valeurs qui ne semblent pas respecter la régle de codification?
liste_sku = df_web['sku'].unique()
liste_sku

array([11862, 16057, 14692, 16295, 15328, 15471, 16515, 16246, nan, 13572,
       16513, 16585, 16269, 15526, 12869, 15575, 11586, 14338, 15425,
       16560, 15361, 13809, 11587, 15022, 14323, 16342, 16029, 15475,
       13754, 14680, 15875, 9636, 13849, 13662, 16564, 13557, 15429,
       14712, 15032, 15481, 15448, 16580, 15441, 804, 15300, 13958, 16071,
       15678, 13895, 15711, 12882, 16053, 13766, 16247, 12640, 15292,
       15476, 15670, 16189, 16038, 14864, 16044, 15324, 15531, 15953,
       15413, 15733, 14366, 15895, 15892, 16472, 15185, 16010, 15793,
       15849, 12315, 15741, 15934, 15148, 15781, 15659, 15106, 15490,
       14507, 14149, 16307, 13736, 14090, 16037, 15758, 14661, 12587,
       15337, 15489, 15201, 16305, 16131, 13435, 15747, 12203, 14509,
       14768, 16262, 14561, 16505, 15717, 16129, 13460, 15871, 15940,
       11602, 13127, 13520, 15480, 13032, 15436, 15269, 15910, 19821,
       16263, 15138, 15146, 15126, 15482, 16186, 13905, 16540, 15856,
       1567

In [42]:
#Si vous avez identifié des codes articles ne respectant pas la régle de codification, consultez-les
X = ~df_web['sku'].isna()
X = df_web['sku'].astype(str)
bonnes_valeurs = X .str.fullmatch(r'\d+')
df_web.loc[~bonnes_valeurs, "sku"]

8                       NaN
20                      NaN
30                      NaN
37                      NaN
41                      NaN
               ...         
1387    bon-cadeau-25-euros
1429                    NaN
1432                    NaN
1445                    NaN
1457                    NaN
Name: sku, Length: 89, dtype: object

In [43]:
#Identifier les lignes sans code article


In [44]:
#Pour les codes articles identifiés, réaliser une analyse et définir l'action à entreprendre


In [45]:
#La clé pour chaque ligne est-elle unique? autrement dit, y a-t-il des doublons?
len(liste_sku), len(df_web['sku'])

(715, 1513)

In [46]:
df_web['sku'].isna().sum()

np.int64(85)

**Avec 715 clés uniques, et plus les 85 valeurs manquantes, on est loin de 1513 lignes, il y a donc bien des clés 'sku' en double.**

In [47]:
#Les lignes sans code article semblent être toutes non renseignées
#Pour s'en assurer, réaliser les étapes suivantes:
#1 - Créer un dataframe avec uniquement les lignes sans code article
df_web_no_sku =  df_web[df_web['sku'].isna()]
df_web_no_sku

,sku,total_sales,tax_status,product_type,post_title,post_excerpt,post_name,guid,post_type,post_mime_type
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#2 - Utiliser la fonction df.info() sur ce nouveau dataframe pour observer le nombre de valeurs renseignées dans chacune des colonnes
df_web_no_sku.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 8 to 1457
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sku             0 non-null      object 
 1   total_sales     2 non-null      float64
 2   tax_status      2 non-null      object 
 3   product_type    2 non-null      object 
 4   post_title      2 non-null      object 
 5   post_excerpt    2 non-null      object 
 6   post_name       2 non-null      object 
 7   guid            2 non-null      object 
 8   post_type       2 non-null      object 
 9   post_mime_type  0 non-null      object 
dtypes: float64(1), object(9)
memory usage: 7.3+ KB


**Je constate que le nouveau dataFrame créé avec les lignes vides pour "sku" donne un Dataframe vide, ainsi, on conclu qu'il faut nécessairement une clé "sku" pour avoir un ligne dans "df_web".**

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.3 - Analyse exploratoire du fichier liaison.xlsx</h3>
</div>

In [49]:
#Dimension du dataset
#Nombre d'observations
df_liaison.shape

(825, 2)

In [50]:
#Nombre de caractéristiques
print(f"Nombre d'observations (lignes) : {df_liaison.shape[0]}")

Nombre d'observations (lignes) : 825


In [51]:
#Consulter le nombre de colonnes
print(f"Nombre d'observations (lignes) : {df_liaison.shape[1]}")

Nombre d'observations (lignes) : 2


In [52]:
#La nature des données dans chacune des colonnes
df_liaison.dtypes

id_web        object
product_id     int64
dtype: object

In [53]:
#Le nombre de valeurs présentes dans chacune des colonnes
df_liaison.count()

id_web        734
product_id    825
dtype: int64

In [54]:
#Le nombre de valeurs manquantes dans chacune des colonnes
df_liaison.isna().sum()

id_web        91
product_id     0
dtype: int64

In [55]:
df_liaison.dropna(subset=["id_web"])

,id_web,product_id
0,15298,3847
1,15296,3849
2,15300,3850
3,19814,4032
4,19815,4039
...,...,...
792,15891,7023
793,15887,7025
822,13127-1,7247
823,14680-1,7329


In [56]:
#Les valeurs de la colonne "product_id" sont-elles toutes uniques?
doublons_product_liaison = df_liaison.duplicated("product_id")
df_liaison[doublons_product_liaison]

,id_web,product_id


**Il y a aucun doublons pour la colonne "product_id"**

In [57]:
#Les valeurs de la colonne "id_web" sont-elles toutes uniques?
doublons_liaison_web = df_liaison.duplicated("id_web", keep = False)
df_liaison[doublons_liaison_web]

,id_web,product_id
19,NaN,4055
49,NaN,4090
50,NaN,4092
119,NaN,4195
131,NaN,4209
...,...,...
817,NaN,7196
818,NaN,7200
819,NaN,7201
820,NaN,7203


In [58]:
#Avons-nous des articles sans correspondance?


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 3 - Jonction des fichiers</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 3.1 - Jonction du fichier df_erp et df_liaison</h3>
</div>

In [59]:
#Fusion des fichiers df_erp et df_liaison


In [60]:
#Y a t-il des lignes ne "matchant" pas entre les 2 fichiers?


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 3.2 - Jonction du fichier df_merge et df_web</h3>
</div>

In [61]:
#Fusionner les datasets df_merge et df_web


In [62]:
#Avons-nous des lignes sans correspondance?


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 4 - Analyse univariée des prix</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 4.1 - Exploration par la visualisation de données</h3>
</div>

In [63]:
#Création d'une boîte à moustache de la répartition des prix grâce à Pandas


In [64]:
#Autre méthode avec plotly express


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 4.2 - Exploration par l'utilisation de méthodes statistiques</h3>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 4.2.1 - Identification par le Z-index</h3>
</div>

In [65]:
#Calculer la moyenne du prix

#Calculer l'écart-type du prix

#Calculer le Z-score


In [66]:
#Quel est le seuil prix dont le z-score est supérieur à 3?


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 4.2.2 - Identification par l'intervalle interquartile</h3>
</div>

In [67]:
#Utilisation de la fonction "describe" de Pandas pour l'étude des mesures de dispersion


In [68]:
#Définir un seuil pour les articles "outliers" en prix


In [69]:
#Définir le nombre d'articles et la proportion de l'ensemble du catalogue "outliers"


In [70]:
#Selon vous, ces outliers sont-ils justifiés ? Comment le démontrer si cela est possible ?


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Etape 5 - Analyse univariée du CA, des quantités vendues, des stocks et de la marge ainsi qu'une analyse multivariée  </h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.1 - Analyse des ventes en CA</h3>
</div>

In [71]:
##############################
# Calculer le CA du site web #
##############################

#Créer une colonne calculant le CA par article

#Calculer la somme de la colonne "ca_par_article"
#Ce résultat correspond au chiffre d'affaire du site web


In [72]:
###############################
# Palmarès des articles en CA #
###############################

#Effectuer le tri dans l'ordre décroissant du CA du dataset df_merge

#Réinitialiser l'index du dataset par un reset_index

#Afficher les 20 premiers articles en CA

#Graphique en barre des 20 premiers articles avec plotly express


In [73]:
#############################
# Calculer le 20 / 80 en CA #
#############################

#Créer une colonne calculant la part du CA de la ligne dans le dataset

#Créer une colonne réalisant la somme cumulative de la colonne précedemment créée

#Grâce aux deux colonnes créées précedemment, calculer le nombre d'articles représentant 80% du CA

#Afficher la proportion que représente ce groupe d'articles dans le catalogue entier du site web


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.2 - Analyse des ventes en quantité</h3>
</div>

In [74]:
#####################################
# Palmarès des articles en quantité #
#####################################

#Effectuer le tri dans l'ordre décroissant de quantités vendues du dataset df_merge

#Réinitialiser l'index du dataset par un reset_index

#Afficher les 20 premiers articles en quantité

#Graphique en barre des 20 premiers articles avec plotly express


In [75]:
#############################
# Calculer le 20 / 80 en CA #
#############################

#Créer une colonne calculant la part en quantité de la ligne dans le dataset

#Créer une colonne réalisant la somme cumulative de la colonne précedemment créée

#Grâce aux deux colonnes créées précedemment, calculer le nombre d'articles représentant 80% des ventes en quantité

#Afficher la proportion que représente ce groupe d'articles dans le catalogue entier du site web


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.3 - Analyse des stocks</h3>
</div>

In [76]:
######################################
# Calculer le nombre de mois de stock #
######################################

#Import de numpy 

#Création de la colonne Rotation de stock

#Remplacement des "inf" par 0

#Effectuer le tri dans l'ordre décroissant du nombre de mois de stock dans le dataset df_merge

#Graphique en barre du flop 20 des produits qui ont le plus de mois de stock

In [77]:
####################################
# Valorisation des stocks en euros #
####################################

#Création de la colonne Valorisation des stocks en euros

#Calculer la somme de la colonne "Valorisation_stock_euros"

In [78]:
##############################################
# Valorisation du nombre de produits en stock #
##############################################

#Calculer la somme de la colonne stock quantity

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.4 - Analyse du taux de marge</h3>
</div>

In [79]:
############################
# Analyse du taux de marge #
############################

#Création de la colonne Prix HT

#Création de la colonne Taux de marge

#Afficher le prix minimum de la colonne "taux_marge"

#Afficher le prix maximum de la colonne "taux_marge"


In [80]:
#Affichage de la ligne avec un taux de marge inférieur à 0


In [81]:
#Création d'un dataframe avec les taux positifs

#Afficher le prix minimum de la colonne "taux_marge"

#Afficher le prix maximum de la colonne "taux_marge"


In [82]:
#Création d'un dataframe avec le taux de marge moyen par type de produit

#Affichage dans un graphique du taux de marge par type de produit


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.5 - Analyse des corrélations entre les variables stock, sales et price</h3>
</div>

In [83]:
############################
# Analyse des corrélations #
############################

#Importation de Seaborn

#Création d'une heatmap de corrélation avec les variables stock, sales et price
#On peut également créer un mask pour n'afficher qu'une demi heatmap

In [84]:
#Que peut-on conclure des corrélations ?

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">Etape 5.6 - Mise à disposition de la nouvelle table sur un fichier Excel</h3>
</div>

In [85]:
#Mettre le dataset df_merge sur un fichier Excel
#Cette étape peut être utile pour partager le résultat du dataset obtenu avec les équipes.  
